<style>div.title-slide {    width: 100%;    display: flex;    flex-direction: row;            /* default value; can be omitted */    flex-wrap: nowrap;              /* default value; can be omitted */    justify-content: space-between;}</style><div class="title-slide">
<span style="float:left;">Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat &amp; Arnaud Legout</span>
<span><img src="media/both-logos-small-alpha.png" style="display:inline" /></span>
</div>


# Opération avancées en pandas

## Complément - niveau intermédiaire

### Introduction

Pandas supporte des opérations de manipulation des Series et DataFrame qui sont similaires dans l'esprit à ce que l'on peut faire avec une base de données et le langage SQL, mais de manière plus intuitive et expressive et beaucoup plus efficacement puisque les opérations se déroulent toutes en mémoire. 

Vous pouvez concaténer (`concat`) des DataFrame, faire des jointures (`merge`), faire des regroupements (`groupby`) ou réorganiser les indexes (`pivot`).

Nous allons dans la suite développer ces différentes techniques. 

### Concaténations avec `concat`

`concat` est utilisé pour concaténer des Series ou des DataFrames. Regardons un exemple.

In [1]:
import numpy as np
import pandas as pd

s1 = pd.Series([30, 35], index=['alice', 'bob'])
s2 = pd.Series([32, 22, 29], index=['bill', 'alice', 'jo'])
pd.concat([s1, s2])

alice    30
bob      35
bill     32
alice    22
jo       29
dtype: int64

On remarque, cependant, que par défaut il n'y a pas de contrôle sur les labels d'indexe dupliqués. On peut corriger cela avec l'argument `verify_integrity` qui va produire une exception s'il y a des labels d'indexes qui ont un recouvrement. Évidemment, cela a un coût de calcul supplémentaire, ça n'est donc à utiliser que si c'est nécessaire.

In [2]:
try:
    pd.concat([s1, s2], verify_integrity=True)
except ValueError as e:
    print(f"erreur de concaténation:\n{e}")

erreur de concaténation:
Indexes have overlapping values: ['alice']


In [3]:
# créons deux series avec les index sans recouvrement
s1 = pd.Series(range(1000), index=[chr(x) for x in range(1000)])
s2 = pd.Series(range(1000), index=[chr(x+2000) for x in range(1000)])

# temps de concaténation avec vérification des recouvrements
%timeit pd.concat([s1, s2], verify_integrity=True)
# temps de concaténation sans vérification des recouvrements
%timeit pd.concat([s1, s2])

565 µs ± 13.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
365 µs ± 6.29 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Par défaut, `concat` concatène les lignes, c'est-à-dire que `s2` sera sous `s1`, mais on peut changer ce comportement un utilisant l'argument `axis`.

In [4]:
p1 = pd.DataFrame(np.random.randint(1, 10, size=(2,2)), columns=list('ab'), index=list('xy'))
p2 = pd.DataFrame(np.random.randint(1, 10, size=(2,2)), columns=list('ab'), index=list('zt'))

# équivalent à pd.concat([p1, p2], axis=0)
# concaténation des lignes
pd.concat([p1, p2])

,a,b
x,9,9
y,4,7
z,1,6
t,5,3


In [5]:
p1 = pd.DataFrame(np.random.randint(1, 10, size=(2,2)), columns=list('ab'), index=list('xy'))
p2 = pd.DataFrame(np.random.randint(1, 10, size=(2,2)), columns=list('cd'), index=list('xy'))

# concaténation des colonnes
pd.concat([p1, p2], axis=1)

,a,b,c,d
x,8,9,9,1
y,8,5,5,8


Regardons maintenant ce cas.

In [6]:
pd.concat([p1, p2])


,a,b,c,d
x,8.0,9.0,NaN,NaN
y,8.0,5.0,NaN,NaN
x,NaN,NaN,9.0,1.0
y,NaN,NaN,5.0,8.0


Vous remarquez que lors de la concaténation, on prend l'union des tous les labels des indexes de `p1` et `p2`, il y a donc des valeurs mises à `NaN`. On peut contrôler ce comportement de plusieurs manières que vous allons voir ci-dessous.

In [7]:
# on concatène les lignes, l'argument join décide quels labels sur l'autre 
# axe on garde (ici sur les colonnes). 

#Par défaut, join utilise la stratégie 'outer', c'est-à-dire 
# qu'on prend la concaténation des labels, si on spécifie 'inner' on prend 
# l'intersection des labels
pd.concat([p1, p2], join='inner')

""
x
y
x
y


In [8]:
# avec join_axes, on peut spécifier les labels qu'on veut garder sous forme 
# d'un objet Index
pd.concat([p1, p2], join_axes=[p1.columns])

,a,b
x,8.0,9.0
y,8.0,5.0
x,NaN,NaN
y,NaN,NaN


In [9]:
pd.concat([p1, p2], join_axes=[pd.Index(['a', 'c'])])

,a,c
x,8.0,NaN
y,8.0,NaN
x,NaN,9.0
y,NaN,5.0


Notons que les Series et DataFrame ont une méthode `append` qui est un raccourci vers `concat`, mais avec moins d'options. 

Pour aller plus loin, voici la documentation officielle : 

http://pandas.pydata.org/pandas-docs/stable/merging.html#concatenating-objects

### Jointures avec `merge`

`merge` est dans l'esprit similaire au `join` en SQL. L'idée est de combiner deux DataFrame en fonction d'un critère d'égalité sur des colonnes. Regardons un exemple.

In [10]:
df1 = pd.DataFrame({'employee': ['Bob', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Sue'],
                    'hire_date': [2004, 2008, 2014]})

In [11]:
print(df1)
print(df2)

  employee        group
0      Bob   Accounting
1     Lisa  Engineering
2      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2      Sue       2014


On souhaite ici combiner `df1` et `df2` de manière à ce que les lignes contenant le même _employee_ soit alignées. Notre critère de merge est donc l'égalité des labels sur la colonne _employee_.

In [12]:
pd.merge(df1, df2)

,employee,group,hire_date
0,Bob,Accounting,2008
1,Lisa,Engineering,2004
2,Sue,HR,2014


Par défaut, `merge` fait un inner join (ou jointure interne) en utilisant comme critère de jointure les colonnes de même nom (ici _employee_). _inner join_ veut dire que pour joindre deux lignes il faut que le même _employee_ apparaisse dans les deux DataFrame. 

Il existe trois type de merges.

 - one-to-one, c'est celui que l'on vient de voir. C'est le merge lorqu'il n'y a pas de labels dupliqués dans les colonnes utilisées comme critère de merge.
 
 - many-to-one, c'est le merge lorsque l'une des deux colonnes contient des labels dupliqués, dans ce cas, on applique la stratégie one-to-one pour chaque lable dupliqué, donc les entrées dupliquées sont préservées.
 
 - many-to-many, c'est la stratégie lorsqu'il y a des entrées dupliquées dans les deux colonnes. Dans ce cas on fait un produit cartésien des lignes. 
 
D'une manière générale, gardez en tête que pandas fait essentiellement ce à quoi on s'attend. Regardons cela sur des exemples.
 
 
 

In [13]:
df1 = pd.DataFrame({'patient': ['Bob', 'Lisa', 'Sue'],
                    'repas': ['SS', 'SS', 'SSR']})
df2 = pd.DataFrame({'repas': ['SS', 'SSR'],
                    'explication': ['sans sel', 'sans sucre']})
print(df1)
print(df2)

  patient repas
0     Bob    SS
1    Lisa    SS
2     Sue   SSR
  explication repas
0    sans sel    SS
1  sans sucre   SSR


In [14]:
# la colonne commune pour le merge est 'repas' et dans une des colonnes 
# (sur df1), il y a des labels dupliqués, on applique la stratégie many-to-one
pd.merge(df1, df2)

,patient,repas,explication
0,Bob,SS,sans sel
1,Lisa,SS,sans sel
2,Sue,SSR,sans sucre


In [15]:
df1 = pd.DataFrame({'patient': ['Bob', 'Lisa', 'Sue'],
                    'repas': ['SS', 'SS', 'SSR']})
df2 = pd.DataFrame({'repas': ['SS', 'SS', 'SSR'],
                    'explication': ['sans sel', 'légumes', 'sans sucre']})
print(df1)
print(df2)

  patient repas
0     Bob    SS
1    Lisa    SS
2     Sue   SSR
  explication repas
0    sans sel    SS
1     légumes    SS
2  sans sucre   SSR


In [16]:
# la colonne commune pour le merge est 'repas' et dans les des colonnes 
# il y a des labels dupliqués, on applique la stratégie many-to-many
pd.merge(df1,df2)

,patient,repas,explication
0,Bob,SS,sans sel
1,Bob,SS,légumes
2,Lisa,SS,sans sel
3,Lisa,SS,légumes
4,Sue,SSR,sans sucre


Dans un merge, on peut contrôler les colonnes à utiliser comme critère de merge. Regardons ces différents cas sur des exemples.

In [17]:
df1 = pd.DataFrame({'employee': ['Bob', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Sue'],
                    'hire_date': [2004, 2008, 2014]})
print(df1)
print(df2)

  employee        group
0      Bob   Accounting
1     Lisa  Engineering
2      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2      Sue       2014


In [18]:
# on décide d'utiliser la colonne 'employee' comme critère de merge
pd.merge(df1, df2, on='employee')

,employee,group,hire_date
0,Bob,Accounting,2008
1,Lisa,Engineering,2004
2,Sue,HR,2014


In [19]:
df1 = pd.DataFrame({'employee': ['Bob', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'HR']})
df2 = pd.DataFrame({'name': ['Lisa', 'Bob', 'Sue'],
                    'hire_date': [2004, 2008, 2014]})
print(df1)
print(df2)

  employee        group
0      Bob   Accounting
1     Lisa  Engineering
2      Sue           HR
   hire_date  name
0       2004  Lisa
1       2008   Bob
2       2014   Sue


In [20]:
# mais on peut également définir un nom de colonne différent 
# à gauche et à droite
m = pd.merge(df1,df2, left_on='employee', right_on='name')
print(m)

  employee        group  hire_date  name
0      Bob   Accounting       2008   Bob
1     Lisa  Engineering       2004  Lisa
2      Sue           HR       2014   Sue


In [21]:
# dans ce cas, comme on garde les colonnes utilisées comme critère dans 
# le résultat du merge, on peut effacer la colonne inutile ainsi
m.drop('name', axis=1)


,employee,group,hire_date
0,Bob,Accounting,2008
1,Lisa,Engineering,2004
2,Sue,HR,2014


`merge` permet également de contrôler la stratégie à appliquer lorsqu'il y a des valeurs dans une colonne utilisée comme critère merge absente dans l'autre colonne. C'est ce que l'on appelle jointure à gauche, jointure à droite, jointure interne (comportement par défaut) et jointure externe. Pour ceux qui ne sont pas familié avec ces notions, regardons des exemples. 

In [22]:
df1 = pd.DataFrame({'name': ['Bob', 'Lisa', 'Sue'],
                    'pulse': [70, 63, 81]})
df2 = pd.DataFrame({'name': ['Eric', 'Bob', 'Marc'],
                    'weight': [60, 100, 70]})
print(df1)
print(df2)

   name  pulse
0   Bob     70
1  Lisa     63
2   Sue     81
   name  weight
0  Eric      60
1   Bob     100
2  Marc      70


In [23]:
# la colonne 'name' est critère de merge dans les deux DataFrame.
# Seul Bob existe dans les deux colonnes. Dans un inner join 
# (le cas par défaut) on ne garde que les lignes pour lesquels il y a une 
# même valeur présente à gauche et à droite
pd.merge(df1, df2) # équivalent à pd.merge(df1, df2, how='inner')

,name,pulse,weight
0,Bob,70,100


In [24]:
# le outer join va au contraire faire une union des lignes et compléter ce 
# qui manque avec NaN
pd.merge(df1, df2, how='outer')

,name,pulse,weight
0,Bob,70.0,100.0
1,Lisa,63.0,NaN
2,Sue,81.0,NaN
3,Eric,NaN,60.0
4,Marc,NaN,70.0


In [25]:
# le left join ne garde que les valeurs de la colonne de gauche 
pd.merge(df1, df2, how='left')

,name,pulse,weight
0,Bob,70,100.0
1,Lisa,63,NaN
2,Sue,81,NaN


In [26]:
# et le right join ne garde que les valeurs de la colonne de droite 
pd.merge(df1, df2, how='right')

,name,pulse,weight
0,Bob,70.0,100
1,Eric,NaN,60
2,Marc,NaN,70


Pour aller plus loin, vous pouvez lire la documentation. Vous verrez notamment que vous pouvez merger sur les indexes (au lieu des colonnes) ou le cas ou vous avez des colonnes de même noms qui ne font pas partie du critère de merge&nbsp;:

http://pandas.pydata.org/pandas-docs/stable/merging.html#database-style-dataframe-joining-merging

### Regroupement avec `groupby`

Regardons maintenant cette notion de groupement. Il s'agit d'une notion très puissante avec de nombreuses options que nous ne couvrirons que partiellement. 
La logique derrière `groupby` est de créer des groupes dans une DataFrame en fonction des valeurs d'une (ou plusieurs) colonne(s), toutes les lignes contenant la même valeur sont dans le même groupe. On peut ensuite appliquer à chaque groupe des opérations qui sont :

 - soit des calculs sur chaque groupe ;
 - soit un filtre sur chaque groupe qui peut garder ou supprimer un groupe ;
 - soit une transformation qui va modifier tout le groupe (par exemple, pour centrer les valeurs sur la moyenne du groupe).
 
Regardons quelques exemples.

In [27]:
d = pd.DataFrame({'key': list('ABCABC'), 'val': range(6)})
print(d)

  key  val
0   A    0
1   B    1
2   C    2
3   A    3
4   B    4
5   C    5


In [28]:
# utilisons comme colonne de groupement 'key'
g = d.groupby('key')
print(g)


`groupby` produit un nouvel objet, mais ne fait aucun calcul. Les calculs seront affectués lors de l'appel de fonction sur ce nouvel objet. Par exemple, calculons la somme pour chaque groupe.

In [29]:
g.sum()

,val
key,
A,3
B,5
C,7


`groupby` peut utiliser comme critère de groupement une colonne, une liste de colonne, ou un index (c'est notamment utile pour les Series). 

On particularité de `groupby` est que le critère de groupement devient un index dans le nouvel objet généré. L'avantage est que l'on a maintenant un accès optimisé sur ce critère, mais l'inconvénient est que sur certaines opérations qui détruise l'index on peut perdre ce critère. On peut contrôler ce comportement avec `as_index`.

In [30]:
g = d.groupby('key', as_index=False)
g.sum()

,key,val
0,A,3
1,B,5
2,C,7


L'objet produit par `groupby` pemet de manipuler les groupes, regardons cela.

In [31]:
d = pd.DataFrame({'key': list('ABCABC'), 'val1': range(6), 'val2' : range(100, 106)})
print(d)

  key  val1  val2
0   A     0   100
1   B     1   101
2   C     2   102
3   A     3   103
4   B     4   104
5   C     5   105


In [35]:
g = d.groupby('key')

# g.groups donne accès au dictionnaire des groupes, les clefs sont le 
# nom du groupe et les valeurs les indexes les lignes appartenant 
# au groupe
g.groups

{'A': Int64Index([0, 3], dtype='int64'),
 'B': Int64Index([1, 4], dtype='int64'),
 'C': Int64Index([2, 5], dtype='int64')}

In [39]:
# Pour accéder directement au groupe, on peut utiliser get_group
g.get_group('A')

,key,val1,val2
0,A,0,100
3,A,3,103


In [42]:
# on peut également filtrer un groupe par colonne lors qu'une 
# opération
g.sum()['val2']

key
A    203
B    205
C    207
Name: val2, dtype: int64

In [41]:
# ou directement sur l'objet produit par groupby
g['val2'].sum()

key
A    203
B    205
C    207
Name: val2, dtype: int64

On peut également itérer sur les groupes avec un boucle for classique

In [44]:
import seaborn as sns
# on charge le fichier de données de pourboires
tips = sns.load_dataset('tips')

# on groupe le DataFrame par jours
g = tips.groupby('day')

# on calcule la moyenne du pourboire par jour
for (group, obj) in g:
    print(f"On {group} the mean tip is {obj['tip'].mean():.3}")


On Thur the mean tip is 2.77
On Fri the mean tip is 2.73
On Sat the mean tip is 2.99
On Sun the mean tip is 3.26


L'objet produit par `groupby` supporte ce que l'on appelle le _dispatch_ de méthodes. Si une méthode n'est pas directement définie sur l'objet produit par `groupby`, elle est appelée sur chaque groupe (il faut donc qu'elle soit définie sur les DataFrame ou les Series). Regardons cela.

In [84]:
# on groupe par jour et on extrait uniquement la colonne 'total_bill'
# pour chaque groupe
g = tips.groupby('day')['total_bill']

# on demande à pandas d'afficher les float avec seulement deux chiffres
# après la virgule
pd.set_option('display.float_format', '{:.2f}'.format)

# on appelle describe() sur g, mais elle n'est pas définie sur ce objet, 
# elle va donc être appelée sur chaque groupe
g.describe()


,count,mean,std,min,25%,50%,75%,max
day,,,,,,,,
Thur,62.00,17.68,7.89,7.51,12.44,16.20,20.16,43.11
Fri,19.00,17.15,8.30,5.75,12.09,15.38,21.75,40.17
Sat,87.00,20.44,9.48,3.07,13.91,18.24,24.74,50.81
Sun,76.00,21.41,8.83,7.25,14.99,19.63,25.60,48.17


In [49]:
# Mais, il y a tout de même un grand nombre de méthodes 
# définies directement sur l'objet produit par le groupby

[x for x in dir(g) if not x.startswith('_')]

['agg',
 'aggregate',
 'all',
 'any',
 'apply',
 'backfill',
 'bfill',
 'corr',
 'count',
 'cov',
 'cumcount',
 'cummax',
 'cummin',
 'cumprod',
 'cumsum',
 'describe',
 'diff',
 'dtype',
 'expanding',
 'ffill',
 'fillna',
 'filter',
 'first',
 'get_group',
 'groups',
 'head',
 'hist',
 'idxmax',
 'idxmin',
 'indices',
 'last',
 'mad',
 'max',
 'mean',
 'median',
 'min',
 'ndim',
 'ngroup',
 'ngroups',
 'nlargest',
 'nsmallest',
 'nth',
 'nunique',
 'ohlc',
 'pad',
 'pct_change',
 'pipe',
 'plot',
 'prod',
 'quantile',
 'rank',
 'resample',
 'rolling',
 'sem',
 'shift',
 'size',
 'skew',
 'std',
 'sum',
 'tail',
 'take',
 'transform',
 'tshift',
 'unique',
 'value_counts',
 'var']

Nous allons regarder la méthode `aggregate` (dont l'alias est `agg`). Cette méthode permet d'appliquer une fonction (ou liste de fonctions) à chaque groupe avec la possibilité d'appliquer une fonction à une colonne spéficique du groupe. 

Une subtilité de `aggregate` et qu'on peut passer soit un objet fonction, soit un nom de fonction sous forme d'une str. Pour l'utilisation du nom fonction marche, il faut que la fonction soit définie sur l'objet produit par le `groupby` ou qu'elle soit définie sur les groupes (donc avec dispatching).

In [52]:
# calculons la moyenne et la variance pour chaque groupe 
# et chaque colonne numérique
tips.groupby('day').agg(['mean', 'std'])

total_bill      tip     size    
           mean std mean std mean std
day                                  
Thur       17.7 7.9  2.8 1.2  2.5 1.1
Fri        17.2 8.3  2.7 1.0  2.1 0.6
Sat        20.4 9.5  3.0 1.6  2.5 0.8
Sun        21.4 8.8  3.3 1.2  2.8 1.0

In [53]:
# de manière équivalente avec les objets fonctions
tips.groupby('day').agg([np.mean, np.std])

total_bill      tip     size    
           mean std mean std mean std
day                                  
Thur       17.7 7.9  2.8 1.2  2.5 1.1
Fri        17.2 8.3  2.7 1.0  2.1 0.6
Sat        20.4 9.5  3.0 1.6  2.5 0.8
Sun        21.4 8.8  3.3 1.2  2.8 1.0

In [55]:
# en appliquant une fonction différente pour chaque colonne,
# on passe alors un dictionnaire qui a pour clef le nom de la 
# colonne et pour valeur la fonction à appliquer à cette colonne
tips.groupby('day').agg({'tip': np.mean, 'total_bill': np.std})

,tip,total_bill
day,,
Thur,2.8,7.9
Fri,2.7,8.3
Sat,3.0,9.5
Sun,3.3,8.8


La méthode `filter` a pour but de filtrer les groupes en fonction d'un critère. Mais attention, `filter` retourne **un sous ensemble des données originale** dans lesquelles les éléments appartenant aux groupes filtrés ont été enlevés.

In [62]:
d = pd.DataFrame({'key': list('ABCABC'), 'val1': range(6), 'val2' : range(100, 106)})
print(d)

  key  val1  val2
0   A     0   100
1   B     1   101
2   C     2   102
3   A     3   103
4   B     4   104
5   C     5   105


In [64]:
# regardons la somme par groupe
d.groupby('key').sum()

,val1,val2
key,,
A,3,203
B,5,205
C,7,207


In [65]:
# maintenant enlevons dans les données originales toutes les lignes
# pour lesquelles la somme de leur groupe est supérieur à 3 
# (ici le groupe A)
d.groupby('key').filter(lambda x: x['val1'].sum()>3)

,key,val1,val2
1,B,1,101
2,C,2,102
4,B,4,104
5,C,5,105


La méthode `transform` a pour but de retourner **un sous ensemble des données originale** dans lesquelles un fonction a été appliquée par groupe. Un usage classique est de centrer des valeurs par groupe ou de remplacer les `NaN` d'un groupe par la valeur moyenne du groupe. 

Attention, `transform` ne doit pas faire de modifications en place, sinon le résultat peut-être faux. Faites donc bien attention de ne pas appliquer des fonctions qui font des modications en place. 

In [66]:
r = np.random.normal(0.5, 2, 4)
d = pd.DataFrame({'key': list('ab'*2), 'data': r,'data2': r*2})
print(d)

   data  data2 key
0   1.2    2.3   a
1   1.8    3.6   b
2   2.4    4.7   a
3   1.7    3.4   b


In [75]:
# je groupe sur la colonne 'key'
g = d.groupby('key')

,data,data2
key,,
a,1.8,3.5
b,1.8,3.5


In [78]:
# maintenant je centre chaque groupe par rapport à sa moyenne
g.transform(lambda x: x - x.mean())

,data,data2
0,-0.6,-1.2
1,0.0,0.1
2,0.6,1.2
3,-0.0,-0.1


Notez que la colonne `key` a disparu, ce comportement est expliqué ici

http://pandas.pydata.org/pandas-docs/stable/groupby.html#automatic-exclusion-of-nuisance-columns

Pour aller plus loin sur `groupby` vous pouvez lire la documentation : 

http://pandas.pydata.org/pandas-docs/stable/groupby.html


### Réorganisation des indexes avec `pivot`

Un manière de voir la notion de pivot est de considérer qu'il s'agit d'une extension de `groupy` à deux dimensions. Pour illustrer cela, prenons un exemple en utilisant le jeux de données seaborn sur les passagers du Titanic. 

In [86]:
titanic = sns.load_dataset('titanic')

In [87]:
# regardons le format de ce jeux de données 
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.00,1,0,7.25,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.00,1,0,71.28,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.00,0,0,7.92,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.00,1,0,53.10,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.00,0,0,8.05,S,Third,man,True,NaN,Southampton,no,True


In [88]:
# regardons maintenant le taux de survie par class et par sex
titanic.pivot_table('survived', index='class', columns='sex')

sex,female,male
class,,
First,0.97,0.37
Second,0.92,0.16
Third,0.50,0.14


Je ne vais pas entrer plus dans le détail, mais vous voyez qu'il s'agit d'un outil très puissant. 

Pour aller plus loin, vous pouvez regarder la documentation officielle

http://pandas.pydata.org/pandas-docs/stable/reshaping.html

mais vous aurez des exemples beaucoup plus parlant en regardant ces examples

https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.09-Pivot-Tables.ipynb

### Gestion des séries temporelles

Il y a un sujet que je n'aborderai pas ici, mais qui est très important pour certains usages, c'est la gestion des séries temporelles. Sachez que pandas supporte des Index spécialisé dans les séries temporelles et que par conséquent toutes les opérations qui consistent à filtrer ou grouper par période de temps sont supporter nativement par pandas. 

Je vous invite de nouveau a regarder la documentation officielle de pandas à ce sujet



### Conclusion

Ce notebook clos notre survol de pandas. C'est un sujet vaste que nous avons déjà largement dégrossi. Pour aller plus loin vous avez évidemment la documentation officielle de pandas

http://pandas.pydata.org/pandas-docs/stable/index.html

mais vous avez aussi l'excellent livre de Jake VanderPlas "Python Data Science Handbook" qui est entièrement disponible sous forme de notebook en ligne 

https://github.com/jakevdp/PythonDataScienceHandbook

Il s'agit d'un très beau travail (c'est rare) utilisant les dernières versions de python, pandas and numpy (c'est encore plus rare) fait par un physicien qui fait de la data science et qui a contribué au développement de nombreux modules de data science en Python.

Pour finir, si vous voulez faire de la data science, il y a un livre incontournable : "An Introduction de Statistical Learning" de G. James, D. Witten, T. Hastie, R. Tibshirani. Ce livre utilise R, mais vous pouvez facilement l'appliquer en utisant pandas.

Les auteurs mettent à disposition gratuitement le PDF du livre ici

http://www-bcf.usc.edu/~gareth/ISL/

N'oubliez pas que si ces ressources vous sont utiles, achetez ces livres pour supporter ces auteurs. Les ressources de grande qualité sont rares, elles demandent un travail énorme à produire, elles doivent être encouragées et recompensée. 
